<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [1]:
import pandas as pd
import sqlite3
import sqlite3 as sq3

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

In [2]:
%%capture
%load_ext sql
%sql sqlite://bike.db

In [3]:
db = sqlite3.connect("bike.db") #Create connection

In [4]:
db

In [5]:
cursor = db.cursor()

In [6]:
pd.read_sql_query("Select * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,SEOUL_BIKE_SHARING,SEOUL_BIKE_SHARING,2,"CREATE TABLE ""SEOUL_BIKE_SHARING"" (\n""DATE"" TE..."
1,table,CITIES_WEATHER_FORECAST,CITIES_WEATHER_FORECAST,173,"CREATE TABLE ""CITIES_WEATHER_FORECAST"" (\n""CIT..."
2,table,BIKE_SHARING_SYSTEMS,BIKE_SHARING_SYSTEMS,178,"CREATE TABLE ""BIKE_SHARING_SYSTEMS"" (\n""COUNTR..."
3,table,WORLD_CITIES,WORLD_CITIES,184,"CREATE TABLE ""WORLD_CITIES"" (\n""CITY"" TEXT,\n ..."


In [7]:
db.close()

In [8]:
seoul_bike_sharing = pd.read_csv("seoul_bike_sharing_clean.csv", parse_dates=["DATE"])
cities_weather_forecast = pd.read_csv("cities_weather_forecast_clean.csv")
bike_sharing_systems = pd.read_csv("bike_sharing_systems_clean.csv")
world_cities = pd.read_csv("world_cities_clean.csv")

### Attribute Information:

- Date : year-month-day
- Rented Bike count - Count of bikes rented at each hour
- Hour - Hour of he day
- Temperature-Temperature in Celsius
- Humidity - %
- Windspeed - m/s
- Visibility - 10m
- Dew point temperature - Celsius
- Solar radiation - MJ/m2
- Rainfall - mm
- Snowfall - cm
- Seasons - Winter, Spring, Summer, Autumn
- Holiday - Holiday/No holiday
- Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [9]:
seoul_bike_sharing.shape

(8465, 14)

In [10]:
seoul_bike_sharing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8465 entries, 0 to 8464
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DATE                   8465 non-null   datetime64[ns]
 1   RENTED_BIKE_COUNT      8465 non-null   int64         
 2   HOUR                   8465 non-null   int64         
 3   TEMPERATURE            8465 non-null   float64       
 4   HUMIDITY               8465 non-null   int64         
 5   WIND_SPEED             8465 non-null   float64       
 6   VISIBILITY             8465 non-null   int64         
 7   DEW_POINT_TEMPERATURE  8465 non-null   float64       
 8   SOLAR_RADIATION        8465 non-null   float64       
 9   RAINFALL               8465 non-null   float64       
 10  SNOWFALL               8465 non-null   float64       
 11  SEASONS                8465 non-null   object        
 12  HOLIDAY                8465 non-null   object        
 13  FUN

## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [11]:
seoul_bike_sharing[seoul_bike_sharing["RENTED_BIKE_COUNT"] == 0]

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [12]:
cities_weather_forecast[:1]

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
0,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [13]:
seoul_bike_sharing["SEASONS"].unique()

array(['Winter', 'Spring', 'Summer', 'Autumn'], dtype=object)

## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [14]:
seoul_bike_sharing["DATE"].min()

Timestamp('2017-01-12 00:00:00')

In [15]:
seoul_bike_sharing["DATE"].max()

Timestamp('2018-12-11 00:00:00')

## Task 6 - Subquery - 'all-time high'

#### Determine which date and hour had the most bike rentals.


### Solution 6


In [16]:
seoul_bike_sharing["RENTED_BIKE_COUNT"].max()

3556

In [17]:
seoul_bike_sharing[seoul_bike_sharing["RENTED_BIKE_COUNT"] == 3556]

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
4770,2018-06-19,3556,18,24.1,57,2.9,1301,15.0,0.56,0.0,0.0,Summer,No Holiday,Yes


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [18]:
seoul_bike_sharing.groupby("SEASONS")[["TEMPERATURE","RENTED_BIKE_COUNT"]].mean()

,TEMPERATURE,RENTED_BIKE_COUNT
SEASONS,,
Autumn,13.821580,924.110480
Spring,13.021685,746.254167
Summer,26.587711,1034.073370
Winter,-2.540463,225.541204


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [19]:
seoul_bike_sharing.groupby("SEASONS")[["RENTED_BIKE_COUNT"]].describe()

RENTED_BIKE_COUNT                                                             
                    count         mean         std  min     25%    50%     75%     max
SEASONS                                                                               
Autumn             1937.0   924.110480  617.547879  2.0  427.00  856.0  1271.0  3298.0
Spring             2160.0   746.254167  618.667962  2.0  225.00  599.0  1118.0  3251.0
Summer             2208.0  1034.073370  690.244759  9.0  526.75  905.5  1442.5  3556.0
Winter             2160.0   225.541204  150.372236  3.0  110.00  203.0   305.0   937.0

Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [20]:
seoul_bike_sharing.groupby("SEASONS").mean()

,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL
SEASONS,,,,,,,,,,
Autumn,924.110480,11.530718,13.821580,59.044915,1.492101,1558.174497,5.150594,0.522783,0.117656,0.0635
Spring,746.254167,11.500000,13.021685,58.758333,1.857778,1240.911574,4.091389,0.680301,0.186944,0.0000
Summer,1034.073370,11.500000,26.587711,64.981431,1.609420,1501.745471,18.750136,0.761255,0.253487,0.0000
Winter,225.541204,11.500000,-2.540463,49.744907,1.922685,1445.987037,-12.416667,0.298181,0.032824,0.2475


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [21]:
world_cities

,CITY,CITY_ASCII,LAT,LNG,COUNTRY,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1.392686e+09
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1.360771e+09
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1.356873e+09
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1.356227e+09
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1.608618e+09
...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1.304218e+09
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1.304206e+09
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1.804043e+09
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1.643739e+09


In [22]:
world_cities[world_cities["CITY"] == "Seoul"]

,CITY,CITY_ASCII,LAT,LNG,COUNTRY,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID
7,Seoul,Seoul,37.5833,127.0,"Korea, South",KR,KOR,Seoul,primary,21794000.0,1.410836e+09


In [23]:
bike_sharing_systems

,COUNTRY,CITY,SYSTEM,BICYCLES
0,Albania,Tirana,NaN,200.0
1,Argentina,Mendoza,NaN,40.0
2,Argentina,"San Lorenzo, Santa Fe",Biciudad,80.0
3,Argentina,Buenos Aires,Serttel Brasil,4000.0
4,Argentina,Rosario,NaN,480.0
...,...,...,...,...
475,United States,"Santa Monica, California",3 Gen. CycleHop and Social Bicycles,500.0
476,United States,"Savannah, Georgia",3 Gen. B-Cycle,16.0
477,United States,"Seattle, Washington",8D,500.0
478,United States,"Spartanburg, South Carolina",3 Gen. B-Cycle,40.0


In [24]:
bike_sharing_systems[bike_sharing_systems["CITY"] == "Seoul"]

,COUNTRY,CITY,SYSTEM,BICYCLES
363,South Korea,Seoul,NaN,20000.0


In [25]:
df_joined = pd.merge(left=world_cities, right=bike_sharing_systems, how='inner',on=["CITY"])

In [26]:
df_joined 

,CITY,CITY_ASCII,LAT,LNG,COUNTRY_x,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID,COUNTRY_y,SYSTEM,BICYCLES
0,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1.356227e+09,India,NaN,NaN
1,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22120000.0,1.156074e+09,China,Forever Bicycle,19165.0
2,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22046000.0,1.076533e+09,Brazil,tembici,600.0
3,Seoul,Seoul,37.5833,127.0000,"Korea, South",KR,KOR,Seoul,primary,21794000.0,1.410836e+09,South Korea,NaN,20000.0
4,Mexico City,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,20996000.0,1.484248e+09,Mexico,3 Gen. Clear CC,6500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Moldava nad Bodvou,Moldava nad Bodvou,48.6156,20.9992,Slovakia,SK,SVK,Košický,NaN,11260.0,1.703801e+09,Slovakia,NaN,25.0
425,Didymóteicho,Didymoteicho,41.3500,26.5000,Greece,GR,GRC,Anatolikí Makedonía kai Thráki,minor,9263.0,1.300999e+09,Greece,3 Gen. Smoove,100.0
426,Seville,Seville,41.0208,-81.8671,United States,US,USA,Ohio,NaN,7812.0,1.840012e+09,Spain,3 Gen. Cyclocity,2100.0
427,Marathónas,Marathonas,38.1550,23.9636,Greece,GR,GRC,Attikí,minor,7170.0,1.300766e+09,Greece,3 Gen.,50.0


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [27]:
df_joined

,CITY,CITY_ASCII,LAT,LNG,COUNTRY_x,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID,COUNTRY_y,SYSTEM,BICYCLES
0,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1.356227e+09,India,NaN,NaN
1,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22120000.0,1.156074e+09,China,Forever Bicycle,19165.0
2,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22046000.0,1.076533e+09,Brazil,tembici,600.0
3,Seoul,Seoul,37.5833,127.0000,"Korea, South",KR,KOR,Seoul,primary,21794000.0,1.410836e+09,South Korea,NaN,20000.0
4,Mexico City,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,20996000.0,1.484248e+09,Mexico,3 Gen. Clear CC,6500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Moldava nad Bodvou,Moldava nad Bodvou,48.6156,20.9992,Slovakia,SK,SVK,Košický,NaN,11260.0,1.703801e+09,Slovakia,NaN,25.0
425,Didymóteicho,Didymoteicho,41.3500,26.5000,Greece,GR,GRC,Anatolikí Makedonía kai Thráki,minor,9263.0,1.300999e+09,Greece,3 Gen. Smoove,100.0
426,Seville,Seville,41.0208,-81.8671,United States,US,USA,Ohio,NaN,7812.0,1.840012e+09,Spain,3 Gen. Cyclocity,2100.0
427,Marathónas,Marathonas,38.1550,23.9636,Greece,GR,GRC,Attikí,minor,7170.0,1.300766e+09,Greece,3 Gen.,50.0


In [28]:
df_joined[(df_joined["BICYCLES"] >= 15000) & (df_joined["BICYCLES"] <= 20000)]

,CITY,CITY_ASCII,LAT,LNG,COUNTRY_x,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID,COUNTRY_y,SYSTEM,BICYCLES
1,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22120000.0,1.156074e+09,China,Forever Bicycle,19165.0
3,Seoul,Seoul,37.5833,127.0000,"Korea, South",KR,KOR,Seoul,primary,21794000.0,1.410836e+09,South Korea,NaN,20000.0
6,Beijing,Beijing,39.9050,116.3914,China,CN,CHN,Beijing,primary,19433000.0,1.156229e+09,China,NaN,16000.0
25,Weifang,Weifang,36.7167,119.1000,China,CN,CHN,Shandong,NaN,9373000.0,1.156913e+09,China,NaN,20000.0
27,Ningbo,Ningbo,29.8750,121.5492,China,CN,CHN,Zhejiang,minor,7639000.0,1.156171e+09,China,NaN,15000.0
68,Zhuzhou,Zhuzhou,27.8407,113.1469,China,CN,CHN,Hunan,minor,3855609.0,1.156042e+09,China,Foshan Tianzhou,20000.0


## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                |
| ---------- | ------- | ------------- | --------------------------------------------------------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                               |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review                               |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins                                  |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation                                                          |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
